## 1 - Mongo DB queries from "companies" dataset.
#### Find succesfull companies in the GAMING related industry (requirements marked in bold above)

- NEED 1: The CEO is vegan = 1 --> vegan restaurants

- NEED 2: The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away = 87 --> vet or pet hairdresser

- NEED 3: Everyone in the company is between 25 and 40, give them some place to go party = 87  --> pubs and clubs

- NEED 4: 30% of the company staff have at least 1 child = 26 --> nurseries or schools

#### - NEED 5: Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design. = 20 --> gaming companies around

- NEED 6: Account managers need to travel a lot = 20 --> airports or train stations

#### - NEED 7: Developers like to be near successful tech startups that have raised at least 1 Million dollars = 15 (Front and Back) --> succesfull tech companies around

### Import libraries

In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np
import json
from pandas import json_normalize
import matplotlib.pyplot as plt

import sys
sys.path.append('../')
import src.offices_functions as of

import warnings
warnings.filterwarnings('ignore')



# https://www.gameindustrycareerguide.com/best-cities-for-video-game-development-jobs/

### MongoCompass connection

In [2]:
client = MongoClient("mongodb://localhost/companies")
db = client.get_database()

In [3]:
list(db.list_collections())

[{'name': 'offices',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('933324c1-5c04-4b84-8815-063d72ebe2fa')},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}},
 {'name': 'companies',
  'type': 'collection',
  'options': {},
  'info': {'readOnly': False,
   'uuid': UUID('f146ed07-dff2-480b-8c8c-5ecafa04d561')},
  'idIndex': {'v': 2, 'key': {'_id': 1}, 'name': '_id_'}}]

In [4]:
db.list_collection_names()

['offices', 'companies']

In [5]:
collection = db.get_collection("companies")
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'companies')

In [6]:
c = collection

### Format offices with several locations

In [7]:
res = c.aggregate([{"$unwind":"$offices"},{"$project":{"_id":0}}])

In [8]:
offices = client.companies.offices
offices.insert_many(res)

In [9]:
res = c.aggregate([
    {"$unwind":"$offices"},
    {"$match":{"offices.latitude":{"$ne":None}, "offices.longitude":{"$ne":None}}}, # # Give only companies where "offices.latitude" and "offices.longitude" is not None
    {"$project":{"_id":0}}])

In [10]:
offices.drop()
offices.insert_many(res)

In [11]:
res = list(offices.find({},{"offices":1}))

### Companies Search

#### Since the company is in the GAMING industry, employees might be wanted to be close to this other type of companies: games_video and companies in tech

In [12]:
c.distinct("category_code")

[None,
 'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'government',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web']

In [13]:
res = list(offices.find({},{"offices":1}))

In [14]:
# select game_video companies
category_1 = {"category_code": "games_video"}
 

# select companies ended in "tech" companies
category_2 = {"category_code":{"$regex": ".*tech.*"}}

              
# give me only companies that total_money_raised is greater than 1M(€/$)"
cond_1 = {"total_money_raised":{"$regex": "M$"}}
              
# give me only companies with "offices.city"
cond_2 = {"offices.city": {"$ne":None}}
cond_3 = {"offices.city": {"$ne":""}}


# we can to make visival the following attributes:
categ_proj = {"_id":1, "name":1, "offices":1, "category_code":1}

In [15]:
affinity_companies = list(offices.find({"$and":[{"$or":[category_1, category_2]}, cond_1, cond_2, cond_3]}, categ_proj).sort("total_money_raised",-1))

In [16]:
affinity_companies[:5]

[{'_id': ObjectId('613517e27c85e5f861db8b05'),
  'name': 'Crocus Technology',
  'category_code': 'cleantech',
  'offices': {'description': 'Headquarters',
   'address1': '710 Lakeway Drive, Suite 100',
   'address2': '',
   'zip_code': '94085',
   'city': 'Sunnyvale',
   'state_code': 'CA',
   'country_code': 'USA',
   'latitude': 37.386774,
   'longitude': -121.987661}},
 {'_id': ObjectId('613517e17c85e5f861db806a'),
  'name': 'Selatra',
  'category_code': 'games_video',
  'offices': {'description': 'Irish Office',
   'address1': '4500 Airport Business Park',
   'address2': 'Kinsale Road',
   'zip_code': '',
   'city': 'Cork',
   'state_code': None,
   'country_code': 'IRL',
   'latitude': 51.8717115,
   'longitude': -8.5010032}},
 {'_id': ObjectId('613517e27c85e5f861db88f2'),
  'name': 'Bigpoint',
  'category_code': 'games_video',
  'offices': {'description': 'Bigpoint Headquarters',
   'address1': 'Drehbahn 47-48',
   'address2': '',
   'zip_code': '20354',
   'city': 'Hamburg',
   

In [17]:
# Convert to a DataFrame
df_affinity_companies = pd.DataFrame(affinity_companies)

In [18]:
df_affinity_companies.head()

,_id,name,category_code,offices
0,613517e27c85e5f861db8b05,Crocus Technology,cleantech,"{'description': 'Headquarters', 'address1': '7..."
1,613517e17c85e5f861db806a,Selatra,games_video,"{'description': 'Irish Office', 'address1': '4..."
2,613517e27c85e5f861db88f2,Bigpoint,games_video,"{'description': 'Bigpoint Headquarters', 'addr..."
3,613517e27c85e5f861db87cc,Prosensa,biotech,"{'description': None, 'address1': None, 'addre..."
4,613517e27c85e5f861db82eb,Owlient,games_video,"{'description': 'Main Office', 'address1': '26..."


In [19]:
# Get the info within office dicts into column

column_dict_keys = ["address1", "address2", "city", "latitude", "longitude", "country_code"]

In [20]:
of.getFromDictColumn("offices", column_dict_keys, df_affinity_companies)

,_id,name,category_code,offices,address1,address2,city,latitude,longitude,country_code
0,613517e27c85e5f861db8b05,Crocus Technology,cleantech,"{'description': 'Headquarters', 'address1': '7...","710 Lakeway Drive, Suite 100",,Sunnyvale,37.386774,-121.987661,USA
1,613517e17c85e5f861db806a,Selatra,games_video,"{'description': 'Irish Office', 'address1': '4...",4500 Airport Business Park,Kinsale Road,Cork,51.871712,-8.501003,IRL
2,613517e27c85e5f861db88f2,Bigpoint,games_video,"{'description': 'Bigpoint Headquarters', 'addr...",Drehbahn 47-48,,Hamburg,53.597305,9.976252,DEU
3,613517e27c85e5f861db87cc,Prosensa,biotech,"{'description': None, 'address1': None, 'addre...",None,None,Leiden,52.158494,4.496199,NLD
4,613517e27c85e5f861db82eb,Owlient,games_video,"{'description': 'Main Office', 'address1': '26...",26-28 rue de ChambÃ©ry,,Paris,48.830809,2.303220,FRA


In [21]:
# Create a subset of companies count by country
df_affinity_companies_top_countries =  df_affinity_companies.groupby(["country_code"] ).size().to_frame(name = "companies_count").reset_index()
df_top3_countries = df_affinity_companies_top_countries.nlargest(3,"companies_count")
df_top3_countries

,country_code,companies_count
22,USA,440
2,CAN,14
10,FRA,11


In [22]:
# Create a subset of companies count by city

df_affinity_companies_top_cities =  df_affinity_companies.groupby(["city", "country_code" ] ).size().to_frame(name = "companies_count").reset_index()
df_top3_cities = df_affinity_companies_top_cities.nlargest(3,"companies_count")
df_top3_cities

,city,country_code,companies_count
195,San Francisco,USA,39
154,New York,USA,27
47,Cambridge,USA,13


#### This resut tell us that the gaming and tech brain is in USA, precisely in San Francisco and New York 

### Select randomly a company address from each city

#### Address in San Franciso

In [23]:
df_affinity_companies_subset = df_affinity_companies[["name", "address1", "city", "latitude", "longitude", "country_code"]]

In [24]:
df_san_francisco_companies = df_affinity_companies_subset[(df_affinity_companies_subset["city"] == "San Francisco") & (df_affinity_companies_subset["country_code"] == "USA")]

In [25]:
df_san_francisco_company = df_san_francisco_companies.sample()
pd.set_option("max_colwidth", 80)
df_san_francisco_company

,name,address1,city,latitude,longitude,country_code
465,Kabam,795 Folsom Street,San Francisco,37.394456,-122.078531,USA


#### Address in New York

In [26]:
df_new_york_companies = df_affinity_companies_subset[(df_affinity_companies_subset["city"] == "New York") & (df_affinity_companies_subset["country_code"] == "USA")]

In [27]:
df_new_york_company = df_new_york_companies.sample()
df_new_york_company

,name,address1,city,latitude,longitude,country_code
434,Stemline Therapeutics,1675 York Avenue,New York,40.777565,-73.945933,USA


In [28]:
df_affinity_companies.to_csv("affinity_companies.csv", index=0)

#### These three cities will meet two of our employees requirementes:
#### - NEED 5: Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design. = 20 --> gaming companies around
#### - NEED 7: Developers like to be near successful tech startups that have raised at least 1 Million dollars = 15 (Front and Back) --> succesfull tech companies around